<a href="https://colab.research.google.com/github/Krishna-coder6111/3d-object/blob/main/cashlesskpis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the excel file through google drive.

In [ ]:
import io
import pandas as pd
from google.colab import drive
import glob
import os
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/My Drive/reports/CP Pilot Due Bill 08-26-2023.xlsx')

Mounted at /content/drive


xlwings is used for:

1.   Automate Excel with Python, such as producing reports
2. Write macros in Python, and run from Excel by clicking on a button
3. Write user-defined functions in Python, and call those function from Excel just like any excel function



In [ ]:
pip install xlwings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.3 MB/s eta 0:00:00


In [ ]:
print(df['Invoice Date'])

0    2023-08-24
1           NaT
2    2023-07-13
3    2023-07-18
4           NaT
        ...    
60   2023-08-24
61          NaT
62   2023-08-26
63          NaT
64          NaT
Name: Invoice Date, Length: 65, dtype: datetime64[ns]


In [ ]:
pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 2.6 MB/s eta 0:00:00


filtering the data based on specifications:

1.   More than 1 due for a customer
2.   Cashless Y
3.   Time past "24 hours" [which is actually 2 days]

In [ ]:
from datetime import datetime, timedelta

# Function to filter the data
def filter_data(df):
    # Convert 'Invoice Date' to datetime
    df['Invoice Date'] = pd.to_datetime(df['Invoice Date'])

    # Identify the correct column for Cashless (either 'Cashless' or 'Cashless?')
    cashless_column = 'Cashless Flag' if 'Cashless Flag' in df.keys() else 'Cashless'

    # Filter based on the conditions: 'Invoice Date', 'Cashless', and 'Due Bill Amount' should be non-negative
    two_days_ago = datetime.now() - timedelta(days=2)
    filtered_df = df[(df['Invoice Date'] < two_days_ago) & (df[cashless_column] == 'Y') & (df['Due Bill Amount'] >= 0)]


    # Function to filter groups with more than one row
    def filter_groups(group):
        return group.shape[0] > 1

    # Group by 'Customer #' and filter groups using the function above
    return filtered_df.groupby('Customer #').filter(filter_groups)

# Find the latest Excel file
path_to_reports = '/content/drive/MyDrive/reports/*.xlsx'
list_of_files = glob.glob(path_to_reports)
latest_file = max(list_of_files, key=os.path.getctime)

# Read all sheet names
xls = pd.ExcelFile(latest_file)
sheet_names = xls.sheet_names


# Create a writer object to save multiple sheets to a new Excel file
output_path = '/content/drive/MyDrive/customers_with_multiple_outstanding_invoices.xlsx'
writer = pd.ExcelWriter(output_path, engine='xlsxwriter')

def insert_total_due_rows(df):
    # Compute the total due bill per 'Customer #'
    total_due = df.groupby('Customer #')['Due Bill Amount'].sum().reset_index()
    total_due['Customer Name'] = 'Total Due' # Add a 'Customer Name' column to identify the total due rows

    # Initialize a list to hold the final rows, including the total due rows
    final_rows = []

    # Iterate through the filtered DataFrame and insert the total due rows
    for customer_number, group in df.groupby('Customer #'):
        group_rows = group.to_dict(orient='records')
        total_due_row = total_due[total_due['Customer #'] == customer_number].to_dict(orient='records')[0]
        final_rows.extend(group_rows + [total_due_row])

    # Convert the final rows to a DataFrame
    final_df = pd.DataFrame(final_rows)
    return final_df

# Loop through the sheets and apply the filtering logic
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for sheet_name in sheet_names:
        df = pd.read_excel(latest_file, sheet_name=sheet_name)
        #filtered_df = filter_data(df)
        # Convert 'Invoice Date' to datetime
        df['Invoice Date'] = pd.to_datetime(df['Invoice Date'])
        # Identify the correct column for Cashless (either 'Cashless' or 'Cashless?')
        cashless_column = 'Cashless Flag' if 'Cashless Flag' in df.keys() else 'Cashless'
        filtered_df = ['hi', 'bye']
        print(filtered_df)

        # Filter based on the conditions: 'Invoice Date', 'Cashless', and 'Due Bill Amount' should be non-negative
        two_days_ago = datetime.now() - timedelta(days=2)
        filtered_df = df[(df['Invoice Date'] < two_days_ago) & (df[cashless_column] == 'Y') & (df['Due Bill Amount'] >= 0)]

        print(filtered_df)


        # Function to filter groups with more than one row
        def filter_groups(group):
            return group.shape[0] > 1

        # Group by 'Customer #' and filter groups using the function above
        filtered_df = filtered_df.groupby('Customer #').filter(filter_groups)

        print(filtered_df)
        #final_df = insert_total_due_rows(filtered_df)
        # Compute the total due bill per 'Customer #'
        total_due = filtered_df.groupby('Customer #')['Due Bill Amount'].sum().reset_index()
        total_due['Customer Name'] = 'Total Due' # Add a 'Customer Name' column to identify the total due rows

        # Initialize a list to hold the final rows, including the total due rows
        final_rows = []

        # Iterate through the filtered DataFrame and insert the total due rows
        for customer_number, group in filtered_df.groupby('Customer #'):
            group_rows = group.to_dict(orient='records')
            total_due_row = total_due[total_due['Customer #'] == customer_number].to_dict(orient='records')[0]
            final_rows.extend(group_rows + [total_due_row])

        # Convert the final rows to a DataFrame
        final_df = pd.DataFrame(final_rows)
        # Convert 'Invoice Date' column back to string format
        final_df['Invoice Date'] = final_df['Invoice Date'].dt.strftime('%m/%d/%Y')

        # Write DataFrame to Excel
        final_df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Get the xlsxwriter workbook and worksheet objects to apply formatting
        workbook  = writer.book
        worksheet = writer.sheets[sheet_name]

        # Create a format for the total row with grey background, bold text, and smaller font
        total_row_format = workbook.add_format({'bg_color': '#D3D3D3', 'bold': True, 'font_size': 11})

        # Apply the format to the total rows
        for row_num, row in final_df.iterrows():
            if row['Customer Name'] == 'Total Due':
                worksheet.set_row(row_num+1, None, total_row_format)

        print(f"Filtered data shape for sheet {sheet_name}: {final_df.shape}") # Print the shape of the filtered data

print(f"File saved to {output_path}")

['hi', 'bye']
Empty DataFrame
Columns: [Business Unit, Sub Market, Pay Lag, BUSS Environment, District #, District Name, Division #, Division Name, Sales Center #, Sale Center Name, Route #, Route Name, Cashless Flag, Customer #, Customer Name, Invoice Date, Invoice #, Sales Amount, ROA Amount, Due Bill Amount, Aging, Action Plan for Sales, Comments, Analyst]
Index: []

[0 rows x 24 columns]
Empty DataFrame
Columns: [Business Unit, Sub Market, Pay Lag, BUSS Environment, District #, District Name, Division #, Division Name, Sales Center #, Sale Center Name, Route #, Route Name, Cashless Flag, Customer #, Customer Name, Invoice Date, Invoice #, Sales Amount, ROA Amount, Due Bill Amount, Aging, Action Plan for Sales, Comments, Analyst]
Index: []

[0 rows x 24 columns]


KeyError: ignored

outputting to an excel file

In [ ]:
print(sheet_names)

['Unpaid Invoices SL', 'Unpaid Invoices Addison', 'Unpaid Invoices Maspeth']


Converting datetime column to a string

In [ ]:
def convert_datetime_to_string(df, column_name='Invoice Date', date_format='%m/%d/%Y'):
    # Convert the specified column to the desired string format
    df[column_name] = df[column_name].dt.strftime(date_format)
    return df

# Convert 'Invoice Date' column back to string format
filtered_df = convert_datetime_to_string(filtered_df)

# Write DataFrame to Excel
writer = pd.ExcelWriter('output_file.xlsx', engine='xlsxwriter')
filtered_df.to_excel(writer, index=False, sheet_name='Sheet1')

# Save the Excel file
writer.save()

NameError: ignored

# Keynotes

1.   Can automate with xlwings
2.   Can improve the data accuracy wih more conditions
3. =@IFS(P9>($A$2-7),"Current",P9>($A$2-14),"Net 7",P9>($A$2-21),"Net 14",P9>($A$2-28),"Net 21",TRUE,"Over 28")


code for non total due that we can use to make pivot tables/dropdown for hierarchy.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import glob
import os

# Function to filter the data
def filter_data(df):
    # Convert 'Invoice Date' to datetime
    df['Invoice Date'] = pd.to_datetime(df['Invoice Date'])

    # Identify the correct column for Cashless (either 'Cashless' or 'Cashless?')
    cashless_column = 'Cashless Flag' if 'Cashless Flag' in df.keys() else 'Cashless'

    # Filter based on the conditions: 'Invoice Date', 'Cashless', and 'Due Bill Amount' should be non-negative
    two_days_ago = datetime.now() - timedelta(days=2)
    filtered_df = df[(df['Invoice Date'] < two_days_ago) & (df[cashless_column] == 'Y') & (df['Due Bill Amount'] >= 0)]

    # Group by 'Customer Name' and filter groups with more than one row
    return filtered_df.groupby('Customer #').filter(lambda group: group.shape[0] > 1)

# Find the latest Excel file
path_to_reports = '/content/drive/MyDrive/reports/*.xlsx'
list_of_files = glob.glob(path_to_reports)
latest_file = max(list_of_files, key=os.path.getctime)

# Read all sheet names
xls = pd.ExcelFile(latest_file)
sheet_names = xls.sheet_names

# Create a writer object to save multiple sheets to a new Excel file
output_path = '/content/drive/MyDrive/raw_for_hierarchy.xlsx'
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for sheet_name in sheet_names:
        df = pd.read_excel(latest_file, sheet_name=sheet_name)
        filtered_df = filter_data(df)

        # Convert 'Invoice Date' column back to string format
        filtered_df['Invoice Date'] = filtered_df['Invoice Date'].dt.strftime('%m/%d/%Y')

        # Write DataFrame to Excel
        filtered_df.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"Filtered data shape for sheet {sheet_name}: {filtered_df.shape}") # Print the shape of the filtered data

print(f"File saved to {output_path}")

Filtered data shape for sheet Unpaid Invoices SL: (0, 24)


KeyError: ignored

In [ ]:
path_to_reports

'/content/drive/MyDrive/reports/*.xlsx'

google email, process file and return output
site -> input file -> output data
excel addon